In [1]:
!pip install scikeras
!pip install shap

  Using cached scikeras-0.12.0-py3-none-any.whl.metadata (4.0 kB)
Using cached scikeras-0.12.0-py3-none-any.whl (27 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 385.1 kB/s eta 0:00:00a 0:00:01
  Using cached slicer-0.0.7-py3-none-any.whl.metadata (3.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.1/457.1 kB 547.8 kB/s eta 0:00:0000:0100:01
Using cached slicer-0.0.7-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 831.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.3 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.1/31.1 MB 3.4 MB/s eta 0:00:0000:0100:01


In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
import shap
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, precision_score, f1_score, accuracy_score, roc_auc_score, roc_curve, auc, recall_score

In [ ]:
# import the training data
# data does not persist after runtime
# everytime the notebook is run we need to re-upload the data
data_raw = pd.read_excel('/content/data.xlsx', index_col=0)

# set up lags for looping
lags = [3, 6, 9, 12, 18]

In [ ]:
# helper function to flatten list of nested lists
# the predicted values are returned like this:
# i.e. [[0],[1],[0],[1]]
# we need to convert it to this:
# i.e. [0,1,0,1]
def flatten_list(matrix_list):
    flat_list = []
    for row in matrix_list:
        flat_list.append(row[0])
    return flat_list

In [ ]:
# set up function to loop over and test different
# thresholds then extract the best one
def find_best_threshold(y_predicted_raw, y_true, flatten_function):
    # create range from 0.4-0.9
    # we can only use range with full integers
    # first step is to make list 40-90 in increments of 5
    # then divide each by 100
    threshold_intervals = [(i / 100) for i in range(40, 91, 5)]
    # now loop over each interval and run the model
    # to see which one performs best
    # create list to store the results
    results = []
    for threshold in threshold_intervals:
        # compare at each threshold
        y_pred_at_threshold = [(1 if pred > threshold else 0) for pred in y_predicted_raw]
        # flatten the results
        #y_pred_at_threshold = flatten_function(y_pred_at_threshold)
        # calcuate the f1 score for the predicted
        threshold_f1 = f1_score(y_true, y_pred_at_threshold)
        # create tuple of each threshold and f1
        results.append((threshold_f1, y_pred_at_threshold, threshold))
    # find the threshold with the max f1
    max_threshold_tuple = max(results, key=lambda x: x[0])
    # now extract the threshold from tuple
    best_threshold = max_threshold_tuple[2]
    y_pred = max_threshold_tuple[1]
    best_f1 = max_threshold_tuple[0]
    # return the result
    return [best_threshold, y_pred, best_f1]

In [ ]:
# create the dirs to export the plots
# then assign the paths to export the direct
# to export the plots
plots_dirs = ['nn-roc-plots', 'nn-shap-plots']
# loop and create
for dir in plots_dirs:
  folder_path = os.path.join('/content/', dir)
  os.makedirs(folder_path, exist_ok=True)
# set up dir paths to export the model plots
# roc curve plots
roc_fig_dir_path = '/content/nn-roc-plots/'
# shap plots
shap_fig_dir_path = '/content/nn-shap-plots/'

In [ ]:
# helper function to properly format the input data
# for the shap package since the format
# is a nested list of lists
def flatten_shap_vals(shap_values):
  # initialize result
  result = []
  # loop and flatten
  for values in shap_values:
    # convert single item inner arrays to one list
    flattened_list = [element[0] for element in values]
    # append result
    result.append(flattened_list)
  # convert to np array
  result = np.array(result)
  return result

In [ ]:
# helper function to perform shap analysis
# used these tutorials
# https://shorturl.at/nwGP8
# don't need to pass in training data for tree models
def shap_analysis(model, X_test, X_train, fig_dir_path, model_name, lag):
    # get the shap explainer variables
    #explainer = shap.DeepExplainer(model, data=X_train)
    explainer = shap.KernelExplainer(model.predict, X_train)
    # get the raw shap values
    shap_values = explainer.shap_values(X_test)
    print('SHAP_VALUES: ', shap_values)
    print('SHAP VALUES SHAPE: ', shap_values.shape)
    # we need to extract the correct values
    # this is a quirk of the nn models' outputs
    # i.e. SHAP VALUES SHAPE:  (1, 6, 1)
    # i.e. SHAP_VALUES:  [[[-1.92598999e-03], ..., [ 6.42130772e-04]]]
    # we need the innermost arrays to be one list
    flat_shap = flatten_shap_vals(shap_values)
    print('FLATTENED SHAP: ', flat_shap)
    print('FLAT SHAP SHAPE: ', flat_shap.shape)
    # create explainer object
    shap_expl = shap.Explanation(flat_shap, base_values=explainer.expected_value, data=X_test)
    # create file name
    file_name = f"beeswarm_plt_{model_name}_lag_{lag}.png"
    # initialize save path
    save_path = os.path.join(fig_dir_path, file_name)
    # create figure object and set size
    plt.figure(figsize=(10, 6))
    # create beeswarm plot
    shap.plots.beeswarm(shap_expl, show=False)
    plt.tight_layout()
    # save the plot
    plt.savefig(save_path, dpi=300)
    # create file name for summary plot
    summary_file_name = f"summary_plot_{model_name}_lag_{lag}.png"
    summary_save_path = os.path.join(fig_dir_path, summary_file_name)
    # create figure for the summary plot as well
    plt.figure(figsize=(10, 6))
    # create summary plot
    shap.summary_plot(flat_shap, X_test, plot_type='bar', show=False)
    plt.tight_layout()
    # export plot
    plt.savefig(summary_save_path, dpi=300)

In [ ]:
# helper function to create roc_curve
def roc_curve_plot(y_true, y_pred_prob, lag, fig_dir_path, model_name):
    # get the fpr, trp, and thresholds
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    # call method on fpr and tpr
    roc_auc = auc(fpr, tpr)
    # create plot
    plt.figure(figsize=(10, 6))
    # we insert the area under curve up to 2 decimal places
    # we can insert using the modulo operator - %
    plt.plot(fpr, tpr, color='lightblue', linewidth=2, label='ROC Curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='darkred', linewidth=2, linestyle=':')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f"Logit ROC Curve for {lag} Month Lag")
    plt.legend(loc="lower right")
    file_name = f"roc_curve_{model_name}_lag_{lag}.png"
    save_path = os.path.join(fig_dir_path, file_name)
    plt.savefig(save_path, dpi=300)
    plt.show()

In [ ]:
# create parameter grid to pass in
# this will help us extract the best model when we do the grid search
param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'optimizer': ['adam', 'rmsprop'],
    'loss': ['binary_crossentropy', 'hinge']
}

In [ ]:
# helper function to start create a new nn model
# creates a neural network with two hidden layers
# we pass this in as the build function
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def neural_network_grid_search(data, lag, test_size, scoring, param_grid):
    """
    Function to run neural network with grid search for hyperparameters
    """

    # make a copy of the original DataFrame to avoid modifying it
    data_copy = data.copy()

    # modify dataset for lag
    data_copy[f"nber_recession_{lag}_month_lag"] = data_copy['nber_recession'].shift(-lag)

    # drop the original recession column and na values
    data_copy = data_copy.drop(columns=['nber_recession'])
    data_copy = data_copy.dropna()

    # set up training and testing data
    X = data_copy.drop(columns=[f"nber_recession_{lag}_month_lag"])
    y = data_copy[f"nber_recession_{lag}_month_lag"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # apply SMOTE only to the training data
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # compile the initial model
    # pass in the starting optimizer and loss functions
    # in order to perform grid search, we need to pass in the model here
    # this tells keras that we are passing in a model that should be
    # used for classification, otherwise, grid search with a neural network
    # becomes very complicated
    model = KerasClassifier(build_fn=create_model, input_dim=X_train_resampled.shape[1], verbose=1)

    # perform grid search
    grid_search_cv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring=scoring, verbose=1)
    grid_result = grid_search_cv.fit(X_train_resampled, y_train_resampled)

    # get best model from the grid search
    best_model = grid_result.best_estimator_
    # also extract the parameters
    best_parameters = grid_result.best_params_

    # get the predicted values
    # returns list of list
    # [chance 0, chance 1]
    y_pred_raw = best_model.predict_proba(X_test)

    # extract the chance of positive classification
    # since the predict_proba returns list of lists
    # of the chance 0 and 1 classification
    # this behaves a bit differently than the other models
    # so we need to use a list comp to extract the
    # positive chances unlike with other models where we
    # can extract like [:,1]
    y_pred_prob_positive = [pred[1] for pred in y_pred_raw]

    # extract the data for the best threshold
    best_model_results = find_best_threshold(y_pred_prob_positive, y_test, flatten_list)
    # extract the relevant results
    threshold = best_model_results[0]
    y_pred = best_model_results[1]
    best_f1 = best_model_results[2]

    # create a confusion matrix to visualize results
    conf_mat = confusion_matrix(y_test, y_pred)

    # make the roc_curve plot
    # this does not take into account
    # the threshold we ind. calculate
    roc_curve_plot(y_test, y_pred_prob_positive, lag, roc_fig_dir_path, 'nn')

    # perform shap analyis on the model
    # we need to unwrap the keras model since KerasKlassifier is not
    # yet supported by the shap package
    # error: <class 'scikeras.wrappers.KerasClassifier'> is not currently a supported model type!
    # unwrapped model
    # reference: https://shorturl.at/rIKY1
    unwrapped_model = best_model.model_
    print('UNWRAPPED MODEL: ', unwrapped_model)
    print('BEST MODEL: ', best_model)
    shap_analysis(unwrapped_model, X_test, X_train_resampled, shap_fig_dir_path, 'nn', lag)

    # get predicted values and metrics
    metrics_obj= {
       'accuracy': accuracy_score(y_test, y_pred),
       'precision': precision_score(y_test, y_pred),
       'recall': recall_score(y_test, y_pred),
       'f1': best_f1,
       'roc_auc': roc_auc_score(y_test, y_pred_prob_positive),
       }

    return {'data': data_copy,
            'best_model': best_model,
            'best_parameters': best_parameters,
            'best_threshold': threshold,
            'y_true': y_test,
            'predicted_vals_binary': y_pred,
            'confusion_matrix': conf_mat,
            'model_metrics': metrics_obj}

In [ ]:
# run model for different lags
# returns tuple of the lag and the results
neural_network_results = [(f"{lag}_month_lag_results", neural_network_grid_search(data_raw, lag, 0.2, 'f1', param_grid)) for lag in lags]

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Epoch 1/10
52/52 [==============================] - 1s 3ms/step - loss: 0.7222 - accuracy: 0.4808
Epoch 2/10
52/52 [==============================] - 0s 3ms/step - loss: 0.6823 - accuracy: 0.5072
Epoch 3/10
52/52 [==============================] - 0s 3ms/step - loss: 0.6778 - accuracy: 0.5180
Epoch 4/10
52/52 [==============================] - 0s 3ms/step - loss: 0.6648 - accuracy: 0.5974
Epoch 5/10
52/52 [==============================] - 0s 3ms/step - loss: 0.6590 - accuracy: 0.6346
Epoch 6/10
52/52 [==============================] - 0s 3ms/step - loss: 0.6401 - accuracy: 0.6947
Epoch 7/10
52/52 [==============================] - 0s 3ms/step - loss: 0.6305 - accuracy: 0.7139
Epoch 8/10
52/52 [==============================] - 0s 8ms/step - loss: 0.6077 - accuracy: 0.7272
Epoch 9/10
52/52 [==============================] - 0s 3ms/step - loss: 0.5957 - accuracy: 0.7452
Epoch 10/10
13/13 [==============================] - 0s 

KeyboardInterrupt: 

In [ ]:
# make a dataframe of all accuracy results
headers_metrics = ['lag', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'threshold', 'conf_matrix']
# store the results for each iteration
iteration_metrics = []
# iterate over results
for result in neural_network_results:
    # extract from the tuple
    metrics = result[1]['model_metrics']
    # extract each value
    values = [val for _, val in metrics.items()]
    # insert name of lag
    values.insert(0, result[0])
    # get the best threshold value for each lag
    threshold = result[1]['best_threshold']
    # append to values
    values.append(threshold)
    # get the confusion matric
    conf_matrix = result[1]['confusion_matrix']
    # append to values
    values.append(conf_matrix)
    # append to the list
    iteration_metrics.append(values)
# convert to a dataframe
metric_data = pd.DataFrame(iteration_metrics, columns=headers_metrics)

print(metric_data)

In [ ]:
# go through and see if the model is over or underestimating recessions
headers_false_true_summary = ['lag', 'recession_true', 'recession_true_pred', 'recession_false', 'recession_false_pred', 'false_pos_rate', 'false_neg_rate']

# store iteration calculations
iteration_summaries = []

# loop over data
for result in neural_network_results:
    # extract the relevant data
    data = result[1]
    y_true_pred = pd.DataFrame({'y_actual': data['y_true'], 'y_predicted': data['predicted_vals_binary']})

    # create row of data with the calculations
    true_pos = np.sum(y_true_pred['y_actual'] == 1)
    true_neg = np.sum(y_true_pred['y_actual'] == 0)
    pred_pos = np.sum(y_true_pred['y_predicted'] == 1)
    false_pos_rate = np.sum((y_true_pred['y_actual'] == 0) & (y_true_pred['y_predicted'] == 1)) / (np.sum(y_true_pred['y_actual'] == 0))
    false_neg_rate = np.sum((y_true_pred['y_actual'] == 1) & (y_true_pred['y_predicted'] == 0)) / (np.sum(y_true_pred['y_actual'] == 1))

    # create a list of the stats to pass in
    summary_stats = [true_pos, pred_pos, true_neg, len(y_true_pred) - pred_pos, false_pos_rate, false_neg_rate]

    # insert lag name
    summary_stats.insert(0, result[0])

    # append to result list
    iteration_summaries.append(summary_stats)

# convert to df
complete_summary_stats = pd.DataFrame(iteration_summaries, columns=headers_false_true_summary)

# print results
print(complete_summary_stats)


In [ ]:
# write data to excel to transfer to local file
# we will do further data processing in another script
path = '/content/nn-results.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')

In [ ]:
# add the metric summary to the excel file
metric_data.to_excel(writer, sheet_name='summary_stats', index=False)

In [ ]:
# loop over the results and export the metrics
for result_data in neural_network_results:
    # extract the relevant data
    sheet_name_start = result_data[0]
    data = result_data[1]
    # create sheet of results that represents actual and predicted vals
    actual_and_predicted = pd.DataFrame({'y_actual': data['y_true'], 'y_predicted': data['predicted_vals_binary']})
    # add data to new sheet
    actual_and_predicted.to_excel(writer, sheet_name=sheet_name_start + '_act_pred', index=False)

In [ ]:
# add summary stats to excel output
complete_summary_stats.to_excel(writer, sheet_name='pos_neg_acc_summary', index=False)
# close writer
writer.close()

In [ ]:
# write the models to another directory we an export
models_dir = '/content/nn-models/'
# create directory if doesn't exist
os.makedirs(models_dir, exist_ok=True)
# loop over the models and extract each of them
for lag, model_data in zip(lags, neural_network_results):
    # write the model to binary and export it using pickle
    with open(os.path.join(models_dir, f"nn-model-{lag}-month-lag.pkl"), "wb") as f:
        # select the model and export
        pickle.dump(model_data[1]['best_model'], f)


In [ ]:
# create zip files of the folders we want to download
!zip -r /content/nn-models.zip /content/nn-models
!zip -r /content/nn-roc-plots.zip /content/nn-roc-plots
!zip -r /content/nn-shap-plots.zip /content/nn-shap-plots

In [ ]:
# import files module to programatically donwload files
from google.colab import files

In [ ]:
# create list of zip folders to download
download_zips = ['nn-models.zip', 'nn-roc-plots.zip', 'nn-shap-plots.zip']
# loop and download
for zip_folder in download_zips:
  # use files module to download
  files.download(zip_folder)